In [72]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection details
postgres_user = 'postgres'
postgres_password = ''
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'football'

# Create a PostgreSQL connection using SQLAlchemy
postgres_uri = f'postgresql://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}'
engine = create_engine(postgres_uri)
df=pd.read_sql_table('combined_data_features', engine)
# View the data
print(df.head())


         Date        Round Venue Result  GF  GA Opponent  xG_x  xGA  Poss  \
0  2023-08-11  Matchweek 1  Away      W   3   0  Burnley   1.9  0.3    65   
1  2023-08-11  Matchweek 1  Away      W   3   0  Burnley   1.9  0.3    65   
2  2023-08-11  Matchweek 1  Away      W   3   0  Burnley   1.9  0.3    65   
3  2023-08-11  Matchweek 1  Away      W   3   0  Burnley   1.9  0.3    65   
4  2023-08-11  Matchweek 1  Away      W   3   0  Burnley   1.9  0.3    65   

   ... early_goals team_pass_ratio  passes_per_game  defensive_work_rate  \
0  ...           1        0.818182         0.236842                  0.0   
1  ...           1        0.818182         0.236842                  0.0   
2  ...           0        0.818182         0.236842                  0.0   
3  ...           0        0.818182         0.236842                  0.0   
4  ...           1        0.818182         0.236842                  0.0   

   formation_width  opponent_formation_width  average_position_x  \
0           

In [73]:
# Check the first few rows of the dataset
df.head()

# Get summary statistics
df.describe()

# Check data types and missing values
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Columns: 113 entries, Date to goals_before_15
dtypes: float64(46), int64(46), object(21)
memory usage: 159.9+ KB


In [64]:
# Separate features (X) and target (y)
X = df.drop('match_result', axis=1)  # Drop the target column
y = df['match_result']  # Assuming match_result is the column to predict


In [65]:
from sklearn.model_selection import train_test_split

# Split the data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting sets
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")


Training set size: (144, 112), Test set size: (37, 112)


In [66]:
# Check the target distribution in the training and test sets
print(f"Training target distribution:\n{y_train.value_counts()}")
print(f"Test target distribution:\n{y_test.value_counts()}")


Training target distribution:
match_result
1    113
0     31
Name: count, dtype: int64
Test target distribution:
match_result
1    30
0     7
Name: count, dtype: int64


In [69]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load your data (replace this with your actual dataframe)
# df = pd.read_csv("your_data.csv")

# Convert Date column to numeric (days since the first match)
df['Date'] = pd.to_datetime(df['Date'])
df['days_since_first_match'] = (df['Date'] - df['Date'].min()).dt.days

# Drop the original Date column
df.drop(columns=['Date'], inplace=True)

# List of categorical columns to encode
categorical_columns = ['Venue', 'Opponent', 'Formation', 'player', 'Result', 'situation', 'shotType', 'Goalkeeper', 'squad', 'season', 'lastAction']

# Apply Label Encoding to each categorical column
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# If you want to apply one-hot encoding instead:
# df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Define features and target
X = df.drop(columns=['match_result'])  # Features (excluding target column)
y = df['match_result']  # Target column



KeyError: 'Date'

In [71]:
# Ensure that all column names are strings
X.columns = X.columns.astype(str)

print(X.columns)
    

Index(['Round', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG_x', 'xGA',
       'Poss', 'Formation',
       ...
       'team_pass_ratio', 'passes_per_game', 'defensive_work_rate',
       'formation_width', 'opponent_formation_width', 'average_position_x',
       'average_position_y', 'goals_after_75', 'goals_before_15',
       'days_since_first_match'],
      dtype='object', length=112)
